In [3]:
import requests
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import copy
import time
import random
import threading
import coinconfig

from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
from binance.helpers import round_step_size
from futures_sign import send_signed_request, send_public_request
from cred import KEY, SECRET

In [4]:
#0:BTCBUSD, 1:BTCUSDT, 2:DOGEUSDT, 3:DOGEBUSD, 4:ETHBUSD, 5:ETHUSDT, 6:EOSUSDT, 7:LTCUSDT, 8:BNBBUSD

In [5]:
client = Client(KEY, SECRET, testnet=False)
config=coinconfig.tradeconfig
coin=coinconfig.coinlist[4]

In [6]:
symbol=coin["symbol"]
asset=coin["asset"]
recvWindow=config["recvWindow"]
dualSidePosition=config["dualSidePosition"]
marginType=config["marginType"]

leverage=coin["leverage"]
roundity=coin["roundity"]
profit_percent=coin["profit_percent"]
diference=coin["diference"]
stop_percent=profit_percent+diference
balance_part=coin["balance_part"]
compensation=coin["compensation"]
ingebit=coin["ingebit"]


In [12]:
def order_clean(symbol):
    orders=client.futures_get_open_orders(symbol=symbol)
    orders_df=pd.DataFrame(orders)
    orders_tp_number=len(orders_df[orders_df['origType']=='STOP'])
    return(orders_tp_number)

In [32]:
orders=client.futures_get_open_orders(symbol=symbol)
orders_df=pd.DataFrame(orders)
orders_sl=orders_df[orders_df['origType']=='STOP']
if len(orders_sl) > 6:
    orders_sl_short=orders_sl[orders_sl['positionSide']=='SHORT']
    orders_sl_long=orders_sl[orders_sl['positionSide']=='LONG']
    if len(orders_sl_short) > 3:
        ids = orders_df[orders_df['origType']=='STOP']['orderId'][-3]
        client.futures_cancel_orders(
            symbol="BTCUSDT", origClientOrderIdList=idsString
        )
    if len(orders_sl_long) > 3:
        ids = orders_df[orders_df['origType']=='STOP']['orderId'][-3]
        client.futures_cancel_orders(
            symbol="BTCUSDT", origClientOrderIdList=idsString
        )
else:
    print('Nothing to clear')

Nothing to clear


In [5]:
client.futures_change_leverage(symbol=symbol, leverage=leverage, recvWindow=recvWindow)

{'symbol': 'ETHBUSD', 'leverage': 50, 'maxNotionalValue': '50000'}

In [6]:
# Margin type
def margin_type_to_isolated():
    status = client.futures_account()
    positions=pd.DataFrame(status['positions'])
    body=positions[positions['symbol']==symbol]['isolated'].astype(bool).tolist()[0]

    if body==True:
        client.futures_change_margin_type(symbol=symbol, marginType=marginType, recvWindow=recvWindow)
        print(f'Changed to {marginType}')
    else:
        print(f'Already {marginType}')

In [7]:
margin_type_to_isolated()

Already CROSSED


In [8]:
# Hedge mode turn on

def hedge_mode_turn_on():
    status=client.futures_get_position_mode() 
    if status['dualSidePosition']==False:
        client.futures_change_position_mode(dualSidePosition=dualSidePosition, recvWindow=recvWindow)
        client.futures_get_position_mode()
        print('Dual turned on')
    else:
        print('Already Dual')
    return(status['dualSidePosition'])

In [9]:
hedge_mode_turn_on()

Already Dual


True

In [10]:
def get_assetbalance(symbol):
    status = client.futures_account()
    balance = pd.DataFrame(status['assets'])
    assetbalance = float(balance.loc[balance['asset'] == asset, 'availableBalance'])
    return(assetbalance)

In [11]:
# Find all opened positions

def get_opened_positions(symbol):
    status = client.futures_account()
    positions=pd.DataFrame(status['positions'])
    open_positions=positions[positions['symbol']==symbol]
    position_long=float(open_positions[open_positions['positionSide']=='LONG']['entryPrice'])
    if position_long>0:
        position=position_long
    else:
        position=float(open_positions[open_positions['positionSide']=='SHORT']['entryPrice'])
    return(position)

In [12]:
get_opened_positions(symbol)

0.0

In [13]:
# Close all orders 

def check_and_close_orders(symbol):
    global isStop 
    a=client.futures_get_open_orders(symbol=symbol)
    if len(a)>0:
        isStop = False
        client.futures_cancel_all_open_orders(symbol=symbol)

In [14]:
def get_symbol_price(symbol):
    prices = client.get_all_tickers()
    df=pd.DataFrame(prices)
    return float(df[ df['symbol']==symbol]['price'])

In [15]:
get_symbol_price(symbol)

1318.28

In [16]:
#получить tick size

def get_tick_size(symbol):
    data = client.futures_exchange_info()
    tick_size = 0.0
    the_symbol = symbol

    found = False
    info = data['symbols']
    for s in range(len(info)):
        if info[s]['symbol'] == the_symbol:
            filters = info[s]['filters']
            for f in range(len(filters)):
                if filters[f]['filterType'] == 'PRICE_FILTER':
                    tick_size = float(filters[f]['tickSize'])
                    found = True
                    break
            break
    return(tick_size)

In [17]:
#Trade config

# maxposition=0.016 #1 = 100% = 1 BTC,
asset_balance=get_assetbalance(symbol)
symbol_price=get_symbol_price(symbol)
maxposition=round(asset_balance*balance_part*leverage/symbol_price,3)
quantity=str(round(maxposition/2,roundity))
tick_size=get_tick_size(symbol)
symbolround=abs(str(tick_size).find('.') - len(str(tick_size))) - 1
quantity

'0.017'

In [18]:
#Stop loss LONG 
def open_sl_long(symbol,quant,tick_size,stop_percent):
    try:
        entry_price=get_opened_positions(symbol)
        stop_price_trigger=str(round(float(entry_price)*(1-stop_percent), symbolround))
        stop_price=str(round(float(stop_price_trigger)*0.9999, symbolround))
        tick=str(tick_size)
        params = {
            "batchOrders": [
                {
                    "symbol":symbol,
                    "side": "SELL",
                    "positionSide": "LONG",
                    "type": "STOP",
                    "quantity": quant,
                    "timeInForce":"GTE_GTC",
                    "tickSize": tick,
                    "stopPrice": stop_price_trigger,
                    "price": stop_price
                }
            ]
        }

        open_position_sl_long_response=send_signed_request('POST', '/fapi/v1/batchOrders', params)
        long_sl_opened=float(open_position_sl_long_response[0]['price'])
        print('LONG sl order opened price:'+str(long_sl_opened))
        return(long_sl_opened)
    
    except :
        print('\n\nSL long not open...')
        

In [19]:
#Take Profit LONG 

def open_tp_long(symbol,quant,tick_size,profit_percent):
    try:
        entry_price=get_opened_positions(symbol)
        take_price_trigger=str(round(float(entry_price)*(1+profit_percent), symbolround))
        take_price=str(round(float(take_price_trigger)*0.9999, symbolround))
        tick=str(tick_size)
        params = {
            "batchOrders": [
                {
                    "symbol":symbol,
                    "side": "SELL",
                    "positionSide": "LONG",
                    "type": "TAKE_PROFIT",
                    "quantity": quant,
                    "timeInForce":"GTE_GTC",
                    "tickSize": tick,
                    "stopPrice": take_price_trigger,
                    "price": take_price
                }
            ]
        }
        open_position_tp_long_response=send_signed_request('POST', '/fapi/v1/batchOrders', params)
        long_tp_opened=float(open_position_tp_long_response[0]['price'])
        print('LONG tp order opened price:'+str(long_tp_opened))
        return(long_tp_opened)
        
    except :
        print('\n\nTP long not open...')
    return(profit_percent,stop_percent)

In [20]:
#Открыть sl&tp LONG если есть позиция

def long_tp_sl_opening(symbol,quant,profit_percent,stop_percent):
    orders=client.futures_get_open_orders(symbol=symbol)
    orders_df=pd.DataFrame(orders)
    if orders_df.empty:
        x = threading.Thread(target=open_tp_long, args=(symbol,quant,tick_size,profit_percent))
        x.start()
        y = threading.Thread(target=open_sl_long, args=(symbol,quant,tick_size,stop_percent))
        y.start()
    else:
        print('LONG SL TP iteration continious..')
    return(profit_percent,stop_percent)

In [21]:
#Открыть позицию LONG 

def open_position_long(symbol,quantity,tick_size,profit_percent,stop_percent):
    sprice=get_symbol_price(symbol)
    close_price=str(round(sprice*(1+0.002),symbolround))
    tick=str(tick_size)
    params = {
        "batchOrders": [
            {
                "symbol":symbol,
                "side": "BUY",
                "positionSide": "LONG",
                "type": "LIMIT",
                "quantity": quantity,
                "timeInForce":"GTC",
                "tickSize": tick,
                "price": close_price        
            }
        ]
    }
    open_position_long_response=send_signed_request('POST', '/fapi/v1/batchOrders', params)
    long_opened=float(open_position_long_response[0]['price'])
    print('LONG order opened price:'+str(long_opened))
    if long_opened>0:
        long_tp_sl=long_tp_sl_opening(symbol,quantity,profit_percent,stop_percent)
    return(long_opened)

In [22]:
#Stop loss SHORT 

def open_sl_short(symbol,quant,tick_size,stop_percent):
    try:
        entry_price=get_opened_positions(symbol)
        stop_price_trigger=str(round(float(entry_price)*(1+stop_percent), symbolround))
        stop_price=str(round(float(stop_price_trigger)*1.0001, symbolround))
        tick=str(tick_size)
        params = {
            "batchOrders": [
                {
                    "symbol":symbol,
                    "side": "BUY",
                    "positionSide": "SHORT",
                    "type": "STOP",
                    "quantity": quant,
                    "timeInForce":"GTE_GTC",
                    "tickSize": tick,
                    "stopPrice": stop_price_trigger,
                    "price": stop_price
                }
            ]
        }
        
        open_position_sl_short_response=send_signed_request('POST', '/fapi/v1/batchOrders', params)
        short_sl_opened=float(open_position_sl_short_response[0]['price'])
        print('SHORT sl order opened price:'+str(short_sl_opened))
        return(short_sl_opened)
    
    except :
        print('\n\nSL short not open...')


In [23]:
#Take Profit SHORT

def open_tp_short(symbol,quant,tick_size,profit_percent):
    try:
        entry_price=get_opened_positions(symbol)
        take_price_trigger=str(round(float(entry_price)*(1-profit_percent), symbolround))
        take_price=str(round(float(take_price_trigger)*1.0001, symbolround))
        tick=str(tick_size)
        params = {
            "batchOrders": [
                {
                    "symbol":symbol,
                    "side": "BUY",
                    "positionSide": "SHORT",
                    "type": "TAKE_PROFIT",
                    "quantity": quant,
                    "timeInForce":"GTE_GTC",
                    "tickSize": tick,
                    "stopPrice": take_price_trigger,
                    "price": take_price
                }
            ]
        }

        open_position_tp_short_response=send_signed_request('POST', '/fapi/v1/batchOrders', params)
        short_tp_opened=float(open_position_tp_short_response[0]['price'])
        print('SHORT tp order opened price:'+str(short_tp_opened))
        return(short_tp_opened)
    
    except :
        print('\n\nTp short not open...')


In [24]:
#Открыть sl&tp SHORT если есть позиция
       
def short_tp_sl_opening(symbol,quant,profit_percent,stop_percent):
    orders=client.futures_get_open_orders(symbol=symbol)
    orders_df=pd.DataFrame(orders)
    if orders_df.empty:
        x = threading.Thread(target=open_tp_short, args=(symbol,quant,tick_size,profit_percent))
        x.start()
        y = threading.Thread(target=open_sl_short, args=(symbol,quant,tick_size,stop_percent))
        y.start()
    else:
        print('SHORT SL TP iteration continious..')

In [25]:
#Открыть позицию SHORT

def open_position_short(symbol,quantity,tick_size,profit_percent,stop_percent):
    sprice=get_symbol_price(symbol)
    close_price=str(round(sprice*(1-0.002),symbolround))
    tick=str(tick_size)
    params = {
        "batchOrders": [
            {
                "symbol":symbol,
                "side": "SELL",
                "positionSide": "SHORT",
                "type": "LIMIT",
                "quantity": quantity,
                "timeInForce":"GTC",
                "tickSize": tick,
                "price": close_price        
            }
        ]
    }
    open_position_short_response=send_signed_request('POST', '/fapi/v1/batchOrders', params)
    short_opened=float(open_position_short_response[0]['price'])
    print('SHORT order opened price:'+str(short_opened))
    if short_opened>0:
        short_tp_sl=short_tp_sl_opening(symbol,quantity,profit_percent,stop_percent)
    return(short_opened)

In [26]:
def get_profit(symbol):
    trades = client.futures_account_trades(symbol=symbol)
    dataset=pd.DataFrame(trades).iloc[-20:]
    bnbcom=dataset[dataset['commissionAsset']=='BNB']['commission'].astype(float).sum()
    busdcom=dataset[dataset['commissionAsset']==asset]['commission'].astype(float).sum()
    com=(bnbcom*270+busdcom)*-1
    real=dataset['realizedPnl'].astype(float).sum()
    profit=float(com+real)
    print(profit)

In [27]:
trades = client.futures_account_trades(symbol=symbol)
dataset=pd.DataFrame(trades).iloc[-20:]
bnbcom=dataset[dataset['commissionAsset']=='BNB']['commission'].astype(float).sum()
busdcom=dataset[dataset['commissionAsset']==asset]['commission'].astype(float).sum()
com=(bnbcom*270+busdcom)*-1
real=dataset['realizedPnl'].astype(float).sum()
profit=float(com+real)
print(profit)
real,com

-0.1124243


(-0.03707, -0.0753543)

In [28]:
get_profit(symbol)

-0.1124243


In [29]:
def get_direction(symbol):
    trades = client.futures_account_trades(symbol=symbol)[-1:]
    dataset = pd.DataFrame(trades)
    resdata = float(dataset['realizedPnl'])
    direction=1
    if resdata < 0:
        direction=0
    return(direction)

In [30]:
get_direction(symbol)

1

In [31]:
str(round(float(quantity)/ingebit,roundity))

'0.009'

In [32]:
def changer(symbol):
    direction=get_direction(symbol)
    f = open("directdoc.txt", "r")
    direc=f.read()
    stop=stop_percent/ingebit
    profit=profit_percent/ingebit
    if direction==0: #реверс
        if direc=='0':
            open_position_long(symbol,str(round(float(quantity)/ingebit,roundity)),tick_size,profit,profit)
            f = open("directdoc.txt", "w")
            f.write("1")
            f.close()
            print('Direction switched from SHORT to LONG')
        else:
            open_position_short(symbol,str(round(float(quantity)/ingebit,roundity)),tick_size,profit,profit)
            print('Direction switched from LONG to SHORT')
            f = open("directdoc.txt", "w")
            f.write("0")
            f.close()
    else: #не меняем
        if direc=='0':
            target=open_position_short(symbol,quantity,tick_size,profit,stop)
            print('Direction SHORT again')
            f = open("directdoc.txt", "w")
            f.write("00")
            f.close()
        if direc=='1':
            target=open_position_long(symbol,quantity,tick_size,profit,stop)
            print('Direction LONG again')
            f = open("directdoc.txt", "w")
            f.write("10")
            f.close()
        if direc=='00':
            quant=str(round(float(quantity)*ingebit,roundity))
            target=open_position_short(symbol,quant,tick_size,profit,stop)
            print('Double SHORT again')
            f = open("directdoc.txt", "w")
            f.write("0")
            f.close()
        if direc=='10':
            quant=str(round(float(quantity)*ingebit,roundity))
            target=open_position_long(symbol,quant,tick_size,profit,stop)
            print('Double LONG again')
            f = open("directdoc.txt", "w")
            f.write("1")
            f.close()

In [33]:
def check_positions_new(symbol):
    positions=get_opened_positions(symbol)
    if positions==0:
        orders=client.futures_get_open_orders(symbol=symbol)
        orders_df=pd.DataFrame(orders)
        if orders_df.empty:
            changer(symbol)

In [34]:
def main(step):
    #getTPSLfrom_telegram()
    try:
        x = threading.Thread(target=check_positions_new(symbol))
        x.start()
        y = threading.Thread(target=get_profit(symbol))
        y.start()   
    except :
        print('\n\nSomething went wrong. Continuing...')

In [35]:
telegram_delay=12
bot_token='5454990445:AAGzfeY80oeI-1su0MEIRVuvTzBYIVlZIXU'
chat_id='92428939'

def getTPSLfrom_telegram():
    strr='https://api.telegram.org/bot'+bot_token+'/getUpdates'
    response = requests.get(strr)
    rs=response.json()
    exit=0

    if(len(rs['result'])>0):
        rs2=rs['result'][-1]
        rs3=rs2['message']
        textt=rs3['text']
        datet=rs3['date']        
        if(time.time()-datet)<telegram_delay:
            if 'exit' in textt:
                exit=1
                telegram_bot_sendtext('Right now! Sir')
            if 'profit' in textt:
                trades = client.futures_account_trades(symbol=symbol)
                dataset=pd.DataFrame(trades).iloc[-50:]
                bnbdata=dataset[dataset['commissionAsset']=='BNB']
                busddata=dataset[dataset['commissionAsset']==asset]
                bnbcom=bnbdata['commission'].astype(float).sum()
                busdcom=busddata['commission'].astype(float).sum()
                com=(bnbcom*270+busdcom)*-1
                real=dataset['realizedPnl'].astype(float).sum()
                profit=com+real
                telegram_bot_sendtext('Profit = ' + str(profit))

    return(exit)

In [36]:
def telegram_bot_sendtext(bot_message):
    bot_token2 = bot_token
    bot_chatID = chat_id
    send_text = 'https://api.telegram.org/bot' + bot_token2 + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + bot_message
    response = requests.get(send_text)
    return response.json()

In [37]:
def prt(message):
    # telegram message
    telegram_bot_sendtext(pointer+': '+message)
    print(pointer+': '+message)

In [38]:
def start_stop():
    starttime=time.time()
    counterr=1
    timeout = time.time() + 60*60*6  # 60 seconds times 60 meaning the script will run for 12 hr
    while time.time() <= timeout:
        try:

            print("script continue running at "+time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
            main(counterr)
            counterr=counterr+1
            # stop=getTPSLfrom_telegram()
            # if stop==1:
            #     break
            if counterr>3:
                # break
                counterr=1
            time.sleep(5 - ((time.time() - starttime) % 5.0)) # 1 minute interval between each new execution
        except KeyboardInterrupt:
            print('\n\KeyboardInterrupt. Stopping.')
        

In [ ]:
start_stop()

script continue running at 2022-11-10 22:37:00
LONG order opened price:1321.36
Double LONG again
LONG sl order opened price:1316.12
-0.12965749999999998
LONG tp order opened price:1318.76
script continue running at 2022-11-10 22:37:05
-0.12965749999999998
script continue running at 2022-11-10 22:37:10
-0.12965749999999998
script continue running at 2022-11-10 22:37:15
-0.12965749999999998
script continue running at 2022-11-10 22:37:20
-0.12965749999999998
script continue running at 2022-11-10 22:37:25
-0.12965749999999998
script continue running at 2022-11-10 22:37:30
-0.12965749999999998
script continue running at 2022-11-10 22:37:35
-0.12965749999999998
script continue running at 2022-11-10 22:37:40
-0.12965749999999998
script continue running at 2022-11-10 22:37:45
LONG order opened price:1321.87
Direction LONG again
-0.1256699


TP long not open...
LONG sl order opened price:1316.93
script continue running at 2022-11-10 22:37:50
-0.1256699
script continue running at 2022-11-10 22:3